# 1. Imports

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
from typing import Optional

import category_encoders
from category_encoders import WOEEncoder
from catboost import CatBoostClassifier
from optuna import Trial
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline

from utils.eda import summary

# 2. Notebook settings

In [19]:
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.2f}'.format

# 3. Data Load

In [20]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
submission_df = pd.read_csv('sample_submission.csv')

# 4. Data Cleaning 

In [21]:
cat_cols = [column for column in train_df.columns if train_df[column].dtype == 'object']
num_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

num_cols.remove('id')
num_cols.remove('CustomerId')
num_cols.remove('IsActiveMember')
num_cols.remove('HasCrCard')
num_cols.remove('Exited')
cat_cols.remove('Surname')
cat_cols.append('IsActiveMember')
cat_cols.append('HasCrCard')

train_df['HasCrCard'] = train_df['HasCrCard'].astype(bool)
train_df['IsActiveMember'] = train_df['IsActiveMember'].astype(bool)
test_df['HasCrCard'] = test_df['HasCrCard'].astype(bool)
test_df['IsActiveMember'] = test_df['IsActiveMember'].astype(bool)
train_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.00,3,0.00,2,True,False,"181,449.97",0
1,1,15749177,Okwudiliolisa,627,France,Male,33.00,1,0.00,2,True,True,"49,503.50",0
2,2,15694510,Hsueh,678,France,Male,40.00,10,0.00,2,True,False,"184,866.69",0
3,3,15741417,Kao,581,France,Male,34.00,2,"148,882.54",1,True,True,"84,560.88",0
4,4,15766172,Chiemenam,716,Spain,Male,33.00,5,0.00,2,True,True,"15,068.83",0


# 5. Model Setup

In [39]:
def instantiate_woe_encoder(trial : Trial) -> WOEEncoder:
  params = {
    'sigma': trial.suggest_float('sigma', 0.001, 5),
    'regularization': trial.suggest_float('regularization', 0, 5),
    'randomized': trial.suggest_categorical('randomized', [True, False])
  }
  return WOEEncoder(**params)

def instantiate_robust_scaler(trial : Trial) -> RobustScaler:
  params = {
    'with_centering': trial.suggest_categorical(
      'with_centering', [True, False]
    ),
    'with_scaling': trial.suggest_categorical(
      'with_scaling', [True, False]
    )
  }
  return RobustScaler(**params)

def instantiate_extra_trees(trial : Trial) -> CatBoostClassifier:
  params = {
    'n_estimators': trial.suggest_int('n_estimators', 50, 200, 10),
    'max_depth': trial.suggest_int('max_depth', 2, 10),
    'random_state': 42
  }
  return CatBoostClassifier(**params)

def instantiate_numerical_pipeline(trial : Trial) -> Pipeline:
  pipeline = Pipeline([
    ('scaler', instantiate_robust_scaler(trial))
  ])
  return pipeline

def instantiate_categorical_pipeline(trial : Trial) -> Pipeline:
  pipeline = Pipeline([
    ('encoder', instantiate_woe_encoder(trial))
  ])
  return pipeline

In [27]:
from sklearn.compose import ColumnTransformer

def instantiate_processor(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> ColumnTransformer:
  
  numerical_pipeline = instantiate_numerical_pipeline(trial)
  categorical_pipeline = instantiate_categorical_pipeline(trial)
  
  processor = ColumnTransformer([
    ('numerical_pipeline', numerical_pipeline, numerical_columns),
    ('categorical_pipeline', categorical_pipeline, categorical_columns)
  ])
  
  return processor

def instantiate_model(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> Pipeline:
  
  processor = instantiate_processor(
    trial, numerical_columns, categorical_columns
  )
  extra_trees = instantiate_extra_trees(trial)
  
  model = Pipeline([
    ('processor', processor),
    ('extra_trees', extra_trees)
  ])
  
  return model

In [28]:
def objective(trial : Trial, X : DataFrame, y : np.ndarray | Series, numerical_columns : Optional[list[str]]=None, categorical_columns : Optional[list[str]]=None, random_state : int=42) -> float:
  if numerical_columns is None:
    numerical_columns = [
      *X.select_dtypes(exclude=['object', 'category']).columns
    ]
  
  if categorical_columns is None:
    categorical_columns = [
      *X.select_dtypes(include=['object', 'category']).columns
    ]
  
  model = instantiate_model(trial, numerical_columns, categorical_columns)
  
  kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
  roc_auc_scorer = make_scorer(roc_auc_score, needs_proba=True)
  scores = cross_val_score(model, X, y, scoring=roc_auc_scorer, cv=kf)
  
  return np.min([np.mean(scores), np.median([scores])])

# 6. Training

In [40]:
from optuna import create_study

study = create_study(study_name='optimization', direction='maximize')

study.optimize(lambda trial: objective(trial, train_df[cat_cols + num_cols], train_df['Exited']), n_trials=100)

[I 2024-01-11 17:41:24,420] A new study created in memory with name: optimization


Learning rate set to 0.399367
0:	learn: 0.5125836	total: 4.41ms	remaining: 790ms
1:	learn: 0.4324972	total: 9.23ms	remaining: 821ms
2:	learn: 0.3900253	total: 13.8ms	remaining: 814ms
3:	learn: 0.3675980	total: 17.5ms	remaining: 771ms
4:	learn: 0.3543321	total: 21.3ms	remaining: 746ms
5:	learn: 0.3474672	total: 25.1ms	remaining: 728ms
6:	learn: 0.3431720	total: 29.9ms	remaining: 738ms
7:	learn: 0.3410476	total: 33.4ms	remaining: 719ms
8:	learn: 0.3383233	total: 37.1ms	remaining: 706ms
9:	learn: 0.3371615	total: 41.5ms	remaining: 705ms
10:	learn: 0.3365119	total: 45.6ms	remaining: 700ms
11:	learn: 0.3359070	total: 49ms	remaining: 686ms
12:	learn: 0.3353752	total: 52.5ms	remaining: 674ms
13:	learn: 0.3346408	total: 56.7ms	remaining: 672ms
14:	learn: 0.3343831	total: 61ms	remaining: 671ms
15:	learn: 0.3337174	total: 65.2ms	remaining: 668ms
16:	learn: 0.3333877	total: 68.8ms	remaining: 659ms
17:	learn: 0.3329842	total: 72.3ms	remaining: 650ms
18:	learn: 0.3327350	total: 77ms	remaining: 652m

[I 2024-01-11 17:41:28,806] Trial 0 finished with value: 0.8768430275176903 and parameters: {'with_centering': True, 'with_scaling': False, 'sigma': 3.2140507061781665, 'regularization': 1.8124802212302393, 'randomized': True, 'n_estimators': 180, 'max_depth': 6}. Best is trial 0 with value: 0.8768430275176903.


157:	learn: 0.3153607	total: 600ms	remaining: 83.5ms
158:	learn: 0.3152811	total: 603ms	remaining: 79.7ms
159:	learn: 0.3151937	total: 608ms	remaining: 76ms
160:	learn: 0.3151071	total: 612ms	remaining: 72.2ms
161:	learn: 0.3150181	total: 616ms	remaining: 68.4ms
162:	learn: 0.3149622	total: 619ms	remaining: 64.6ms
163:	learn: 0.3148677	total: 624ms	remaining: 60.8ms
164:	learn: 0.3147591	total: 628ms	remaining: 57.1ms
165:	learn: 0.3147318	total: 632ms	remaining: 53.3ms
166:	learn: 0.3146496	total: 636ms	remaining: 49.5ms
167:	learn: 0.3145810	total: 640ms	remaining: 45.7ms
168:	learn: 0.3144232	total: 644ms	remaining: 41.9ms
169:	learn: 0.3143407	total: 648ms	remaining: 38.1ms
170:	learn: 0.3142871	total: 652ms	remaining: 34.3ms
171:	learn: 0.3141994	total: 656ms	remaining: 30.5ms
172:	learn: 0.3140938	total: 660ms	remaining: 26.7ms
173:	learn: 0.3139892	total: 663ms	remaining: 22.9ms
174:	learn: 0.3139237	total: 667ms	remaining: 19.1ms
175:	learn: 0.3138368	total: 671ms	remaining: 15

[I 2024-01-11 17:41:33,423] Trial 1 finished with value: 0.8756362000720518 and parameters: {'with_centering': True, 'with_scaling': True, 'sigma': 4.491531280734782, 'regularization': 4.89264905899418, 'randomized': True, 'n_estimators': 190, 'max_depth': 6}. Best is trial 0 with value: 0.8768430275176903.


...

[I 2024-01-11 17:42:35,520] Trial 22 finished with value: 0.8889799547059919 and parameters: {'with_centering': False, 'with_scaling': False, 'sigma': 1.1416943376299817, 'regularization': 2.570633462949739, 'randomized': False, 'n_estimators': 100, 'max_depth': 4}. Best is trial 12 with value: 0.8890464123150345.


67:	learn: 0.3156463	total: 198ms	remaining: 93.1ms
68:	learn: 0.3155691	total: 201ms	remaining: 90.3ms
69:	learn: 0.3154725	total: 204ms	remaining: 87.4ms
70:	learn: 0.3154028	total: 206ms	remaining: 84.3ms
71:	learn: 0.3153423	total: 209ms	remaining: 81.3ms
72:	learn: 0.3152893	total: 213ms	remaining: 78.6ms
73:	learn: 0.3152453	total: 216ms	remaining: 76ms
74:	learn: 0.3151647	total: 220ms	remaining: 73.2ms
75:	learn: 0.3151109	total: 222ms	remaining: 70.3ms
76:	learn: 0.3150377	total: 226ms	remaining: 67.4ms
77:	learn: 0.3149785	total: 228ms	remaining: 64.4ms
78:	learn: 0.3149327	total: 231ms	remaining: 61.4ms
79:	learn: 0.3147949	total: 234ms	remaining: 58.6ms
80:	learn: 0.3147562	total: 237ms	remaining: 55.6ms
81:	learn: 0.3147024	total: 240ms	remaining: 52.6ms
82:	learn: 0.3145982	total: 243ms	remaining: 49.7ms
83:	learn: 0.3145608	total: 245ms	remaining: 46.7ms
84:	learn: 0.3145188	total: 248ms	remaining: 43.8ms
85:	learn: 0.3144710	total: 251ms	remaining: 40.9ms
86:	learn: 0.3

[I 2024-01-11 17:42:37,390] Trial 23 finished with value: 0.8883580325359436 and parameters: {'with_centering': False, 'with_scaling': False, 'sigma': 2.7983653946966225, 'regularization': 1.0664703963621431, 'randomized': False, 'n_estimators': 70, 'max_depth': 3}. Best is trial 12 with value: 0.8890464123150345.


Learning rate set to 0.5
0:	learn: 0.4802120	total: 4.05ms	remaining: 442ms
1:	learn: 0.4016205	total: 7.67ms	remaining: 414ms
2:	learn: 0.3641938	total: 11.9ms	remaining: 423ms
3:	learn: 0.3453868	total: 15.7ms	remaining: 415ms
4:	learn: 0.3377345	total: 19.5ms	remaining: 409ms
5:	learn: 0.3307033	total: 23.8ms	remaining: 412ms
6:	learn: 0.3274343	total: 28.2ms	remaining: 415ms
7:	learn: 0.3257980	total: 31.7ms	remaining: 404ms
8:	learn: 0.3249325	total: 35.4ms	remaining: 397ms
9:	learn: 0.3243372	total: 38.8ms	remaining: 388ms
10:	learn: 0.3235238	total: 42.6ms	remaining: 383ms
11:	learn: 0.3231059	total: 46.1ms	remaining: 376ms
12:	learn: 0.3225016	total: 50.3ms	remaining: 375ms
13:	learn: 0.3219213	total: 54.2ms	remaining: 371ms
14:	learn: 0.3217017	total: 57.6ms	remaining: 365ms
15:	learn: 0.3214865	total: 61.7ms	remaining: 362ms
16:	learn: 0.3212679	total: 65.6ms	remaining: 359ms
17:	learn: 0.3209364	total: 70.5ms	remaining: 360ms
18:	learn: 0.3205871	total: 76.5ms	remaining: 366

[I 2024-01-11 17:42:40,501] Trial 24 finished with value: 0.888683160546428 and parameters: {'with_centering': False, 'with_scaling': False, 'sigma': 1.178121700669067, 'regularization': 2.5275470091455876, 'randomized': False, 'n_estimators': 110, 'max_depth': 5}. Best is trial 12 with value: 0.8890464123150345.


# 7. Model evaluation

In [ ]:
study.best_params
best_trial = study.best_trial

In [ ]:
model = instantiate_model(best_trial, num_cols, cat_cols)
model.fit(train_df[cat_cols + num_cols], train_df['Exited'])

In [ ]:
# Check for underfitting/overfitting by analyzing splits
# folds = range(1, kf.get_n_splits() + 1)
# plt.plot(folds, rmse_train, 'o-', color='green', label='train')
# plt.plot(folds, rmse_test, 'o-', color='red', label='test')
# plt.legend()
# plt.grid()
# plt.xlabel('Number of fold')
# plt.ylabel('mean squared error')
# plt.show()

In [ ]:
test_probabilities = model.predict_proba(test_df[cat_cols + num_cols])[:, 1]

In [11]:
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub['Exited'] = test_probabilities
sub = sub.set_index('id')
sub.head()

NameError: name 'test_probabilities' is not defined

In [ ]:
sub.to_csv('submission.csv')